In [2]:
import os

In [3]:
!pwd

/home/jovyan/notebooks/executions


In [4]:
import sys
sys.path.append('/home/jovyan/')

In [5]:
from experiments.reporting.report_repository import ReportRepository
from experiments.slack_client import SlackNotifier
from neat.neat_logger import get_neat_logger
from neat.population_engine import EvolutionEngine
from neat.reporting.reports_pyneat import EvolutionReport
from neat.utils import timeit
import math
import random
from itertools import count
import numpy as np

from experiments.logger import logger
from experiments.slack_client import Notifier
from neat.configuration import get_configuration
from neat.evaluation.evaluation_engine import EvaluationStochasticEngine
from neat.evolution_operators.crossover import Crossover
from neat.evolution_operators.mutation import Mutation
from neat.genome import Genome
from neat.reporting.reports_pyneat import EvolutionReport
from neat.species import SpeciationEngine
from neat.stagnation import Stagnation
from neat.population_engine import PopulationEngine
from neat.utils import timeit
import neat
import tests
from config_files.configuration_utils import create_configuration

In [6]:
# dir(neat)

In [7]:
LOGS_PATH = f'{os.getcwd()}/'
logger = get_neat_logger(path=LOGS_PATH)

### Configuration

In [8]:
config_file = 'titanic'
config = create_configuration(filename=f'/{config_file}.json')

In [9]:
# TODO: better mechanism for override
config.pop_size = 150
# config.n_samples = 50
#
config.n_generations = 300
config.node_add_prob = 0.5
config.n_output = 2
config.n_processes = 10


ALGORITHM_VERSION = 'bayes-neat'
DATASET = config_file
# CORRELATION_ID = 'parameters_grid'
CORRELATION_ID = 'test'

In [10]:
report_repository = ReportRepository.create(project='neuro-evolution', logs_path=LOGS_PATH)
notifier = SlackNotifier.create(channel='batch-jobs')

In [11]:
# evaluation_engine.n_processes

### Initialize

In [12]:
report = EvolutionReport(report_repository=report_repository,
                         algorithm_version=ALGORITHM_VERSION,
                         dataset=DATASET,
                         correlation_id=CORRELATION_ID)

In [13]:
population_engine = PopulationEngine(stagnation_engine=Stagnation())
speciation_engine = SpeciationEngine()
evaluation_engine = EvaluationStochasticEngine()

In [14]:
end_condition = 'normal'
# try:
# initialize population
population = population_engine.initialize_population()
speciation_engine.speciate(population, generation=0)

population = evaluation_engine.evaluate(population=population)

# report
report.report_new_generation(generation=0,
                                  population=population,
                                  species=speciation_engine.species)

2020-03-22 11:22:07,874 - INFO - Generation 0. Best fitness: -1.981. N-Parameters Best: 8
2020-03-22 11:22:07,875 - INFO -                          Mean fitness: -2.936. Mean N-Parameters: 8.0
2020-03-22 11:22:07,876 - INFO -     New best individual (60) found with fitness -1.981
Confusion Matrix:
[[12000  5600]
 [ 3849  7151]]
Accuracy: 66.96153846153847 %


In [15]:
def run_generation(generation, population):
    
    # create new generation's population
    population = population_engine.reproduce(species=speciation_engine.species,
                                             pop_size=population_engine.pop_size,
                                             generation=generation)
    # create new species based on new population
    speciation_engine.speciate(population, generation=generation)

    # evaluate
    population = evaluation_engine.evaluate(population=population)

    # generation report
    report.report_new_generation(generation=generation,
                                 population=population,
                                 species=speciation_engine.species)
    generation += 1
    return population

In [16]:
def show_species(species):
    species_keys = list(species.keys())
    representative_fitnesses = [round(specie.representative.fitness, 3) for specie in species.values()]
    best_fitnesses = [round(specie.fitness, 3) if specie.fitness else None for specie in species.values()]

    n_genomes = [len(specie.members) for specie in species.values()]
    
    print(f'      keys: {species_keys}')
    print(f'      fitnesses: {representative_fitnesses}')
    print(f'      best fitness: {best_fitnesses}')

    print(f'      n_members: {n_genomes}')

In [17]:
generation = 0

In [18]:
for i in range(50):
    generation += 1
    population = run_generation(generation, population)
    show_species(species=speciation_engine.species)

2020-03-22 11:22:13,635 - INFO - Generation 1. Best fitness: -1.204. N-Parameters Best: 14
2020-03-22 11:22:13,636 - INFO -                          Mean fitness: -2.215. Mean N-Parameters: 10.093
2020-03-22 11:22:13,637 - INFO -     New best individual (234) found with fitness -1.204
Confusion Matrix:
[[ 7120 10480]
 [ 4370  6630]]
Accuracy: 48.07692307692308 %
      keys: [1, 2]
      fitnesses: [-2.708, -2.659]
      best fitness: [-1.981, -2.757]
      n_members: [99, 51]
2020-03-22 11:22:16,552 - INFO - Generation 2. Best fitness: -0.771. N-Parameters Best: 16
2020-03-22 11:22:16,553 - INFO -                          Mean fitness: -1.715. Mean N-Parameters: 14.253
2020-03-22 11:22:16,553 - INFO -     New best individual (400) found with fitness -0.771
Confusion Matrix:
[[17600     0]
 [11000     0]]
Accuracy: 61.53846153846154 %
      keys: [1, 2]
      fitnesses: [-2.077, -2.818]
      best fitness: [-1.204, -1.474]
      n_members: [88, 62]
2020-03-22 11:22:19,854 - INFO - Gener

In [33]:
show_species(species=speciation_engine.species)

      keys: [1, 2]
      fitnesses: [-1.402, -0.747]
      n_members: [69, 81]


In [19]:
speciation_engine.species[1].representative.fitness

-1.640533208847046

In [20]:
speciation_engine.species[2].representative.fitness

-3.7541580200195312

### Check evaluation results

In [31]:
from neat.evaluation.evaluate_simple import evaluate_genome
from neat.evaluation.utils import get_dataset
from neat.loss.vi_loss import get_loss, get_beta

In [29]:
testing = False

In [33]:
dataset = get_dataset(config.dataset, testing=testing)
loss = get_loss(problem_type=config.problem_type)

GETTING DATASET
